In [14]:
! pip install mistralai




# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-hybrid-gen/dataset2.jsonl


In [15]:
import os
from mistralai import Mistral

api_key = "QsVtSj8rL1TZVL0l2usBvNPk0dF7bOMZ"
model = "mistral-large-latest"

client = Mistral(api_key=api_key)


In [16]:
import json
import random

# File paths
input_file = "/kaggle/input/dataset-hybrid-gen/dataset2.jsonl"
training_file = "/kaggle/working/training_data.jsonl"
validation_file = "/kaggle/working/validation_data.jsonl"

# Read the dataset
with open(input_file, "r") as f:
    lines = f.readlines()

random.shuffle(lines)

# Split into training (80%) and validation (20%)
split_idx = int(0.9 * len(lines))
training_lines = lines[:split_idx]
validation_lines = lines[split_idx:]

# Save training and validation datasets
with open(training_file, "w") as f_train:
    f_train.writelines(training_lines)

with open(validation_file, "w") as f_valid:
    f_valid.writelines(validation_lines)

print("Training and validation datasets created successfully!")


Training and validation datasets created successfully!


In [17]:
training_data = client.files.upload(
    file={
        "file_name": training_file,
        "content": open(training_file, "rb"),
    }
)  
print("Training File ID:", training_data.id)


Training File ID: 6f272358-3f85-4ed6-a6b4-dd65eb0f6028


In [18]:
validation_data = client.files.upload(
    file={
        "file_name": validation_file,
        "content": open(validation_file, "rb"),
    }
)  
print("Validation File ID:", validation_data.id)


Validation File ID: 9d5d6b11-8d20-49e6-87f7-7c146ef589a1


In [19]:
import time

# Function to check job status
def get_job_details(job_id):
    return client.fine_tuning.jobs.get(job_id=job_id)

def check_job_status(job_id):
    job_status = client.fine_tuning.jobs.get(job_id=job_id).status
    return job_status

# Create the fine-tuning job
created_jobs = client.fine_tuning.jobs.create(
    model="open-mistral-7b",
    training_files=[{"file_id": training_data.id, "weight": 1}],
    validation_files=[validation_data.id],
    hyperparameters={
        "training_steps": 10,
        "learning_rate": 0.0001
    },
    auto_start=True
)

# Poll for job status until it is VALIDATED or SUCCESS
job_id = created_jobs.id
while True:
    job_details = get_job_details(job_id)
    status = check_job_status(job_id)
    print(f"Job Status: {status}")
    if status == "SUCCESS":
        break
    elif status in ["FAILED_VALIDATION", "CANCELLED"]:
        # Print job details to understand the failure
        print("Job failed. Details:", job_details)
        raise Exception(f"Job {job_id} has status {status}")
    time.sleep(10)  # Wait for 60 seconds before checking again

# Retrieve the fine-tuned model ID
fine_tuned_model_id = job_details.fine_tuned_model

# Print the fine-tuned model ID
print(f"Fine-tuned model ID: {fine_tuned_model_id}")



Job Status: QUEUED
Job Status: RUNNING
Job Status: RUNNING
Job Status: RUNNING
Job Status: RUNNING
Job Status: RUNNING
Job Status: RUNNING
Job Status: RUNNING
Job Status: RUNNING
Job Status: RUNNING
Job Status: RUNNING
Job Status: SUCCESS
Fine-tuned model ID: ft:open-mistral-7b:c1b1fb9b:20241207:ac1ac042


In [20]:


chat_response = client.chat.complete(
    model=fine_tuned_model_id,
    messages = [{"role":'user', "content":'I am a 18-year-old male with a BMI of 16.83. My fitness goal is weight gain. I am interested in muscular fitness and currently at a underweight level. I have no hypertension and no diabetes. What exercises and diet would you recommend?'}]
)

print(chat_response.choices[0].message.content)



For your goal of weight gain, I recommend the following exercises: Squats, Bench Press, Overhead Press, Deadlifts, and Pull-ups. For your diet, consider consuming plenty of protein such as beans, peas, lentils, nuts and seeds, milk and cheese, and eggs. Additionally, include foods rich in carbohydrates like wholegrains, potatoes, rice and pasta. Also consider vegetables, fruits, and a good source of healthy fat like olive oil, rapeseed oil, and avocados. Stay hydrated by drinking plenty of water. Ensure you consult a healthcare professional or Registered Dietitian before starting any new exercise or diet plan. Also, monitor your progress and adjust your plan accordingly.


In [21]:
test_prompt = "I am a 25-year-old male looking to gain muscle. What should I eat?"
response = client.chat.complete(
    model=fine_tuned_model_id,
    messages = [{"role":"user","content":test_prompt}]
)
print(response.choices[0].message.content)


To gain muscle, you'll want to focus on consuming nutritious foods that are rich in protein, carbohydrates, and healthy fats. Here are some food suggestions:

1. Protein: Eggs, lean cuts of meat (chicken, turkey, and fish), and plant-based proteins like beans, lentils, and tofu.
2. Carbohydrates: Brown rice, quinoa, oatmeal, sweet potatoes, and whole grain bread.
3. Healthy Fats: Avocados, nuts, seeds, and olive oil.
4. Vegetables: Green leafy veggies like spinach, kale, and broccoli, as well as other colorful vegetables like bell peppers, carrots, and tomatoes.

It's also important to stay hydrated by drinking plenty of water throughout the day. Make sure to monitor your intake and adjust your diet accordingly, based on your individual needs and goals. Additionally, having a balanced diet, consistent exercise, and adequate sleep are also essential for muscle growth.
